In [1]:
import librosa
import sounddevice as sd

def load_audio(file_path):
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    return audio_data, sample_rate

def play_audio(audio_data, sample_rate):
    sd.play(audio_data, sample_rate)
    sd.wait()  # Wait until the sound has finished playing

# Example usage
file_path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Audio_transcription_files\test\tmp1gjsg3lv.mp3'
audio_data, sample_rate = load_audio(file_path)
play_audio(audio_data, sample_rate)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_7204\2094738530.py:5: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sample_rate = librosa.load(file_path, sr=None)
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [ ]:
# Loading the libraries
from datasets import load_dataset
from transformers import pipeline
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess
import librosa
import numpy as np

from IPython.display import Audio as IPythonAudio

In [ ]:
# Load the MP3 file
# audio_path = r'C:\Users\Administrator\Desktop\LLM_work\SenseVoiceSmall\example\zh.mp3'
audio_path = r'C:\Users\Administrator\Desktop\Backend-LLM\audio_text_llm\SenseVoiceSmall\audio_sample\听故事学中文 Learn Chinese with 12 Stories - The Easiest Way to Improve Chinese.mp3'
audio_array, sample_rate = librosa.load(audio_path, sr=None)

In [ ]:
# Print details
print("Sample Rate:", sample_rate)
print("Audio Array:", audio_array)
print("Audio Array datatype is Array: ", isinstance(audio_array,np.ndarray) )

In [ ]:
# Resampling the Audio so it matches to the requirement of the model
audio_16KHz = librosa.resample(audio_array,
                               orig_sr=sample_rate,
                               target_sr=16000)

In [ ]:
model_dir = "FunAudioLLM/SenseVoiceSmall"

# SenseVoiceSmall 


In [ ]:
# model_dir = r'C:/Users/Administrator/Desktop/LLM_work/SenseVoiceSmall'
# pretrained_model_path=  r'C:\Users\Administrator\Desktop\LLM_work\SenseVoiceSmall\model.pt'
model = AutoModel(
    model='iic/SenseVoiceSmall',
    # init_param = pretrained_model_path
)

# en
res = model.generate(
    input=audio_16KHz,
    cache={},
    language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)

# paraformer-zh

In [ ]:
model = AutoModel(
    model='paraformer-zh',  # This is the directory of model path
    device="cuda:0",  # "cuda:0" for GPU (if CUDA is available) or "cpu" for CPU.
    hub="hf",   # "hf" for Hugging Face Hub, "local" for local filesystem.
)

# en
res = model.generate(
    input=audio_16KHz,
    cache={},
    language="zn",
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)



# ct-punc

In [ ]:
# This model is used to detect the punctuation in the generated text
model = AutoModel(model="ct-punc", model_revision="v2.0.4")
res = model.generate(input="那今天的会就到这里吧 happy new year 明年见")
print(res)

# fsmn-vad

In [ ]:
from funasr import AutoModel

model = AutoModel(model="fsmn-vad", model_revision="v2.0.4")

res = model.generate(input=audio_16KHz)
print(res)

# Combined all

In [ ]:
model = AutoModel(
    model='iic/SenseVoiceSmall',
     vad_model="fsmn-vad",
     vad_kwargs={"max_single_segment_time": 30000},
    device="cpu",
    #punc_model = "ct-punc"
)

# en
res = model.generate(
    input=audio_16KHz,
    cache={},
    language="zn",  # "zn", "en", "yue", "ja", "ko", "nospeech"
    use_itn=True,
    batch_size_s=60,
    merge_vad=True,  #
    merge_length_s=15,
)
text = rich_transcription_postprocess(res[0]["text"])
print(text)

In [ ]:
# Save the transcription to a text file
with open("transcription_SenseVoiceSmall-fsmn-vad.txt", "w", encoding="utf-8") as file:
    file.write(text)

# Intensity Checking

In [1]:
import numpy as np
import librosa
import logging

def text_generator(model, audio_sample):
    text = model.generate(
        input=audio_sample,
        cache={},
        language="auto",
        use_itn=True,
        batch_size_s=60,
        merge_vad=False
    )
    return text
    
def compute_intensity(segment):
    """Compute the intensity of a given audio segment as RMS."""
    return np.sqrt(np.mean(segment**2))

def filter_high_intensity_segments(audio_array, segment_duration: float, intensity_threshold: float):
    """Filter high-intensity audio segments from the given audio array."""
    segment_samples = int(segment_duration * 16000)  # Number of samples per segment
    high_intensity_segments = []

    # Split audio into segments and process their intensities
    num_segments = len(audio_array) // segment_samples
    for i in range(num_segments):
        start_sample = i * segment_samples
        end_sample = start_sample + segment_samples
        segment = audio_array[start_sample:end_sample]
        
        # Calculate intensity for each segment
        intensity = compute_intensity(segment)
        print(f"Segment {i+1} Intensity: {intensity:.6f}")
        
        # Only keep the segment if its intensity exceeds the threshold
        if intensity > intensity_threshold:
            logging.info(f"Keeping high-intensity segment {i+1} with intensity {intensity:.6f}")
            high_intensity_segments.append(segment)
        else:
            logging.info(f"Skipping low-intensity segment {i+1} with intensity {intensity:.6f}")

    # If there are remaining samples that don't form a full segment, calculate their intensity
    if len(audio_array) % segment_samples != 0:
        segment = audio_array[num_segments * segment_samples:]
        intensity = compute_intensity(segment)
        print(f"Last Segment Intensity: {intensity:.6f}")
        if intensity > intensity_threshold:
            logging.info(f"Keeping last high-intensity segment with intensity {intensity:.6f}")
            high_intensity_segments.append(segment)

    return high_intensity_segments

def audio_to_text_model_offline(audio_file_path: str, model, intensity_threshold: float = 0.00) -> str:
    """Converts audio file to text using the provided model, processing only high-intensity segments."""
    try:
        # Load the audio file
        audio_array, sampling_rate = librosa.load(audio_file_path, sr=None)
        audio_array = librosa.resample(audio_array, orig_sr=sampling_rate, target_sr=16000)

        # Define the duration of each segment (for example, 2 seconds)
        segment_duration = 2.0  # seconds

        # Filter high-intensity segments
        high_intensity_segments = filter_high_intensity_segments(audio_array, segment_duration, intensity_threshold)

        # Concatenate high-intensity segments into a single array
        if high_intensity_segments:
            combined_audio = np.concatenate(high_intensity_segments)
            logging.info("Transcribing combined audio of length: %d", len(combined_audio))
            print(combined_audio)

            # Check if the length of the combined audio is greater than 30 seconds
            if len(combined_audio) > (30 * 16000):
                transcriptions = []
                sample_length = int(30 * 16000)
                for start in range(0, len(combined_audio), sample_length):
                    end = min(start + sample_length, len(combined_audio))
                    chunk = combined_audio[start:end]
                    generated_text = text_generator(model, chunk)
                    transcriptions.append(generated_text[0]["text"])
                
                # Join all transcriptions
                transcription_result = " ".join(transcriptions)
            else:
                # Transcribe the combined audio
                res = model.generate(
                    input=combined_audio,
                    vad_model="fsmn-vad",
                    cache={},
                    language="auto",
                    use_itn=True,
                    batch_size_s=60,
                    merge_vad=False
                )
                transcription_result = generated_text[0]["text"]
        else:
            logging.info("No high-intensity segments to transcribe.")
            transcription_result = ""

        return transcription_result.strip()
    
    except Exception as e:
        logging.error("Error in audio_to_text_model_offline: %s", e)
        return ""


In [2]:
from funasr import AutoModel
path = r'C:\Users\Administrator\Desktop\Backend-Algorithm-LLM\Algorithm\audio_text_llm\SenseVoiceSmall\audio_sample\How to Learn Chinese Faster and Smarter.mp3'

model = AutoModel(
    model='paraformer-zh',  # This is the directory of model path
    device="cuda:0",  # "cuda:0" for GPU (if CUDA is available) or "cpu" for CPU.
    hub="hf",   # "hf" for Hugging Face Hub, "local" for local filesystem.
)


transcription_result = audio_to_text_model_offline(path, model)
print(transcription_result)


<unknown>:815: SyntaxWarning: invalid escape sequence '\w'
<unknown>:1398: SyntaxWarning: invalid escape sequence '\w'
<unknown>:815: SyntaxWarning: invalid escape sequence '\w'
<unknown>:1398: SyntaxWarning: invalid escape sequence '\w'
<unknown>:815: SyntaxWarning: invalid escape sequence '\w'
<unknown>:1398: SyntaxWarning: invalid escape sequence '\w'
<unknown>:815: SyntaxWarning: invalid escape sequence '\w'
<unknown>:1398: SyntaxWarning: invalid escape sequence '\w'


New version available: 1.1.12. Your current version is 1.1.2.
Please use the command "pip install -U funasr" to upgrade.


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 10 files: 100%|██████████| 10/10 [00:00<?, ?it/s]
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\funasr\train_utils\load_pretrained_model.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no l

Segment 1 Intensity: 0.186192
Segment 2 Intensity: 0.141330
Segment 3 Intensity: 0.125173
Segment 4 Intensity: 0.163063
Segment 5 Intensity: 0.153528
Segment 6 Intensity: 0.125310
Segment 7 Intensity: 0.089459
Segment 8 Intensity: 0.163529
Segment 9 Intensity: 0.155053
Segment 10 Intensity: 0.167562
Segment 11 Intensity: 0.136328
Segment 12 Intensity: 0.128469
Segment 13 Intensity: 0.161070
Segment 14 Intensity: 0.170396
Segment 15 Intensity: 0.134508
Segment 16 Intensity: 0.140540
Segment 17 Intensity: 0.106716
Segment 18 Intensity: 0.105363
Segment 19 Intensity: 0.098633
Segment 20 Intensity: 0.137254
Segment 21 Intensity: 0.136791
Segment 22 Intensity: 0.135722
Segment 23 Intensity: 0.158884
Segment 24 Intensity: 0.137166
Segment 25 Intensity: 0.085068
Segment 26 Intensity: 0.142252
Segment 27 Intensity: 0.116888
Segment 28 Intensity: 0.105967
Segment 29 Intensity: 0.139438
Segment 30 Intensity: 0.129457
Segment 31 Intensity: 0.113237
Segment 32 Intensity: 0.115851
Segment 33 Intens

  0%|          | 0/1 [00:00<?, ?it/s]c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\funasr\models\paraformer\model.py:251: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(False):
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\funasr\models\paraformer\cif_predictor.py:212: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(False):
rtf_avg: 0.026: 100%|██████████| 1/1 [00:00<00:00,  1.36it/s]                                                                                          

大家好如果你想快速的掌握中文词汇请继续观看这个视频picture this you just spend hours and hours learning hundreds of new chinese words and now you're sitting down to chat in chinese with innative speaker in the space were about thirty seconds all of the new vovocahave you thought you have just burned into your memeory seems eevaporate from your brain and you left tongue tide and struggling to say anything beyond even a basic你好吗if you've ever had a similar experience of struggling to memorize and then actually use new chinese words and you're looking for a complete system to supercharge your vovocab learning then this video was made for you today i'll go over the best system i found so far for effectively paroritising vomeorizing and then actually mobilizing new chinese vocabulary that you've learned t will be from a recomination ation of my own personal experience self studying mandaring alalongside my medical degree ccambritips from experts ts have written books about language joinning theory and also evidence 

In [16]:
transcription_result

''

In [3]:
import numpy as np
import sounddevice as sd
import librosa

# Set the sampling rate
sampling_rate = 16000  # Sample rate in Hz

def audio_callback(indata, frames, time, status):
    """Callback function to process the audio data."""
    if status:
        print(status)
    # Convert the input data to a 1D array
    audio_data = indata[:, 0]  # Assuming mono audio
    # Calculate intensity as the root mean square (RMS) of the audio samples
    intensity = np.sqrt(np.mean(audio_data**2))  # RMS calculation
    print(f'Intensity: {intensity:.6f}')

# Start recording from the microphone
with sd.InputStream(callback=audio_callback, channels=1, samplerate=sampling_rate):
    print("Recording... Press Ctrl+C to stop.")
    try:
        # Keep the program running indefinitely
        while True:
            pass
    except KeyboardInterrupt:
        print("Stopped recording.")


Recording... Press Ctrl+C to stop.
Intensity: 0.000014
Intensity: 0.000604
Intensity: 0.006211
Intensity: 0.000958
Intensity: 0.000437
Intensity: 0.000220
Intensity: 0.000151
Intensity: 0.000172
Intensity: 0.000162
Intensity: 0.000139
Intensity: 0.000117
Intensity: 0.000095
Intensity: 0.000094
Intensity: 0.000080
Intensity: 0.000082
Intensity: 0.000071
Intensity: 0.000107
Intensity: 0.000094
Intensity: 0.000088
Intensity: 0.000093
Intensity: 0.000089
Intensity: 0.000085
Intensity: 0.000105
Intensity: 0.000118
Intensity: 0.000141
Intensity: 0.000139
Intensity: 0.000130
Intensity: 0.000133
Intensity: 0.000150
Intensity: 0.000172
Intensity: 0.000200
Intensity: 0.000214
Intensity: 0.000112
Intensity: 0.000106
Intensity: 0.000089
Intensity: 0.000087
Intensity: 0.000097
Intensity: 0.000135
Intensity: 0.000163
Intensity: 0.000085
Intensity: 0.000099
Intensity: 0.000121
Intensity: 0.000118
Intensity: 0.000110
Intensity: 0.000152
Intensity: 0.000377
Intensity: 0.000204
Intensity: 0.000124
Inten